In [1]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date


In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



chattanooga_table_id = "opendatadbt.311.chattanooga311"

#Inital Load
chattanooga311 = requests.get("https://www.chattadata.org/resource/8qb9-5fja.json?$limit=10000&$where=created_date >= '2021-03-22'")
chattanooga311json = chattanooga311.json()
chattanooga_df = pd.json_normalize(chattanooga311json)
chattanooga_df_parsed = chattanooga_df['publiclocation.coordinates'].apply(pd.Series)
chattanooga_df_cleaned = pd.concat([chattanooga_df.drop(['publiclocation.coordinates'], axis=1), chattanooga_df_parsed], axis=1)
mapping = {chattanooga_df_cleaned.columns[-1]:'latitude', chattanooga_df_cleaned.columns[-2]:'longitude',chattanooga_df_cleaned.columns[-3]:'publiclocationtype'}
chattanooga_df_cleaned = chattanooga_df_cleaned.rename(columns=mapping)
chattanooga_df_cleaned = chattanooga_df_cleaned.drop(['publiclocation.type'], axis=1)
cols = ['service_request_key','created_date','completed_at','department','request_type','status_code','latitude','longitude']
chattanooga_df_cleaned = pd.DataFrame(chattanooga_df_cleaned,columns = cols)
chattanooga_df_cleaned

In [3]:
#Incremental load
chattanooga_table_id = "opendatadbt.311.chattanooga311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(created_date), max(completed_at)
FROM `opendatadbt.311.chattanooga311`
limit 10
"""

chattanoogamaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
chattanoogamaxopendatedate = str(chattanoogamaxopendatedataframe['f0_'][0])

chattanoogamaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
chattanoogamaxclosedate = str(chattanoogamaxclosedatedataframe['f1_'][0])



In [4]:
#Incremental Load

chattanooga311 = requests.get("https://www.chattadata.org/resource/8qb9-5fja.json?$limit=50000&$where=created_date>'"+chattanoogamaxopendatedate+"' OR completed_at between '" +chattanoogamaxclosedate+ "' AND '" + current_datetime + "'")
chattanooga311json = chattanooga311.json()
chattanooga_df = pd.json_normalize(chattanooga311json)
chattanooga_df_parsed = chattanooga_df['publiclocation.coordinates'].apply(pd.Series)
chattanooga_df_cleaned = pd.concat([chattanooga_df.drop(['publiclocation.coordinates'], axis=1), chattanooga_df_parsed], axis=1)
mapping = {chattanooga_df_cleaned.columns[-1]:'latitude', chattanooga_df_cleaned.columns[-2]:'longitude',chattanooga_df_cleaned.columns[-3]:'publiclocationtype'}
chattanooga_df_cleaned = chattanooga_df_cleaned.rename(columns=mapping)
chattanooga_df_cleaned = chattanooga_df_cleaned.drop(['publiclocationtype'], axis=1)
cols = ['service_request_key','created_date','completed_at','department','request_type','status_code','latitude','longitude']
chattanooga_df_cleaned = pd.DataFrame(chattanooga_df_cleaned,columns = cols)
chattanooga_df_cleaned

,service_request_key,created_date,completed_at,department,request_type,status_code,latitude,longitude
0,9768868,2021-03-25T08:19:34.000,2021-03-26T23:42:45.000,PW - Solid Waste,Missed Garbage,O-CLOSED,NaN,NaN
1,9767965,2021-03-24T20:22:45.000,2021-03-26T23:43:33.000,PW - Solid Waste,Missed Garbage,O-CLOSED,35.079370,-85.168614
2,9781274,2021-03-27T10:20:43.000,NaN,NaN,Bulk Trash,O-NEW,35.121386,-85.222305
3,9781958,2021-03-27T14:23:47.000,NaN,NaN,Loose Leaf Collection,O-NEW,35.157095,-85.211508
4,9765095,2021-03-24T13:33:32.000,2021-03-26T23:44:31.000,PW - Solid Waste,Missed Garbage,O-CLOSED,35.063290,-85.229575
...,...,...,...,...,...,...,...,...
134,9783026,2021-03-27T23:29:04.000,NaN,NaN,Street Lights (not Traffic Signals),O-NEW,NaN,NaN
135,9782213,2021-03-27T16:16:49.000,NaN,NaN,Bagged Yard Waste,O-NEW,NaN,NaN
136,9783041,2021-03-27T23:46:58.000,NaN,NaN,Brush Collection,O-NEW,NaN,NaN
137,9781373,2021-03-27T11:04:06.000,NaN,NaN,Traffic Calming,O-NEW,NaN,NaN


In [5]:
chattanooga_table_id = "opendatadbt.311.chattanooga311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('chattanooga311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    chattanooga_df_cleaned, table_ref, job_config=job_config
).result()  # Make an API request.
